In [178]:
from urllib import request
from bs4 import BeautifulSoup
import json

num = 0
title_list = []; writer_list = []; genre_list = []; category_list = []; introduction_list = []

def getLists(url):
    global num
    html = request.urlopen(url).read()
    soup = BeautifulSoup(html,'html.parser')
    jsondata = json.loads(soup.text)
    
    for i in jsondata['data']:
        t = i.get('title')
        if(t in title_list):
            continue
        else:
            title_list.append(t)
    
            writers = []
            for j in i.get('cartoon').get('artists'):
                writers.append(j.get('name'))
            writer_list.append(writers)
    
            genres = []
            for j in i.get('cartoon').get('genres'):
                genres.append(j.get('name'))
            genre_list.append(genres)
    
            categories = []
            for j in i.get('cartoon').get('categories'):
                categories.append(j.get('name'))
            category_list.append(categories)
    
            introduction_list.append(i.get('introduction'))
    
            num = num + 1

url_mon = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/mon'
url_tue = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/tue'
url_wed = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/wed'
url_thu = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/thu'
url_fri = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/fri'
url_sat = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/sat'
url_sun = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/sun'
url_finished = 'http://webtoon.daum.net/data/pc/webtoon/list_daily_ranking/finished'
url_recommend = 'http://webtoon.daum.net/data/pc/webtoon/list_recommend'

url_list = [url_mon, url_tue, url_wed, url_thu, url_fri, url_sat, url_sun, url_finished, url_recommend]

for i in url_list:
    getLists(i)

In [179]:
import pandas as pd

import pandas as pd

webtoonfile = pd.DataFrame({'제목': title_list, \
                            '작가': writer_list, \
                            '장르': genre_list, \
                            '카테고리': category_list, \
                            '한줄 스토리': introduction_list})

webtoonfile.to_csv('daum_webtoon_crawling.csv')

In [180]:
webtoonfile.head()

,제목,작가,장르,카테고리,한줄 스토리
0,무지개다리 파수꾼,"[이서, 이서]",[드라마],"[반려동물, 동물, 의학, 스토리]","돈과 성공만을 전부로 알던 유명 수의사 한철은, 어느 날 동물들의 소리를 들을 수 ..."
1,서천화원,"[곡, 마하로]",[순정],"[궁, 썸, 의학, 스토리]",자신의 운명을 스스로 개척하기 위해 궁을 벗어난 공주님의 좌충우돌 모험 로맨스
2,봄날의 팔광,"[우표, 우표]","[순정, 판타지]","[연애, 전래동화, 사랑, 스토리]","'해와 달' 속 달님, 천방지축 선녀와 지상 최대 싸가지 남자가 펼치는 판타지 로맨스!"
3,뱀파이어님의 대리식사,"[웬즈데이, 멜리]","[순정, 판타지]","[뱀파이어, 동거, 먹방, 스토리]",대학생 시영이와 외로운 뱀파이어의 심쿵 먹방 동거생활!
4,현세이의 보이는 라디오,"[부족, 이제환]","[드라마, 일상]","[질병, 사랑, 성장드라마, 학원, 스토리]",스트리머 현세이의 일상 소통 이야기
